<div align="center">
<h1 style="color:#2c3e50;">🧠 Tesis Final de Master</h1>
<h2 style="color:#34495e;">Evaluación Sesgos en Compas</h2>
<h3 style="color:#34495e;">ANÁLISIS EXTENSO DEL DATASET TWOYEARS EN BÚSQUEDA DE ERRORES SUSCEPTIBLES DE VARIAR RESULTADOS ESTADÍSTICOS</h3>

<img src="https://www.shutterstock.com/image-photo/businessman-searching-information-technology-ai-600nw-2529559953.jpg" width="800">

**Nuclio Digital School**  
*TFM curso presencial marzo 2025*

---

**Autor:** Azahara Bravo, Daniel y María
**Fecha:** 20 de Junio, 2025  
**Versión:** 1.0  

</div>

<hr style="border:none; height:2px; background:linear-gradient(to right, #3498db, #e74c3c, #f39c12);">

<hr style="border:none; height:1px; background:#ddd;">

<div style="text-align:center; margin:20px 0;">
• • • •
</div>

# 0. **INTRODUCCIÓN AL PROYECTO**

Para crear el conjunto de datos TwoYears Propublica usó esta lógica:
- Entran personas que tengan Compas realizado almenos dos años antes del fin del período de recopilación de datos en teoría el 31/12/16 pero vemos que existe otro(30/03/16).
- Se descuenta el tiempo en prisión o cárcel a esas personas. 
- Entran personas que desde Compas a reincidencia pasan 2 años. 

Recordando los datos del dataset RawCompas es importante recordad que la fecha por la que se incluyen datos allí es que la Evaluación Compas esté realizada entre el 1/01/2013 y el 31/12/2014. 

Revisando la bibliografía encontrada en relación a la controversia sobre el dataset de Propublica Twoyears vemos que la problemática se centra en la homogéneidad de los datos que dista de hacer a esta muestra estadísticamente correcta. 

En este notebook queremos analizar qué criterios reales forman este dataset, ya que después de leer el estudio de  Barestein y dar una ojeada en las fechas del dataset TwoYears publicado por propublica nuestra atención fué llamada por casos demasiado extremos incluidos. 

Durante el notebook vamos a observar datos que nos dejaron algo perplejos y que distaban de la supuesta lógica de Propublica, vamos a analizar cómo son esas excepciones y como influye la retirada de esos datos del dataset. Además como estamos de acuerdo con el criterio de Barestein hemos decidido crear nuestro Twoyears_clean uniendo nuestros descubrimientos a su corte. 

Vamos a indagar un poco.

<hr style="border:none; height:2px; background:linear-gradient(to right, #3498db, #e74c3c, #f39c12);">

<hr style="border:none; height:1px; background:#ddd;">

<div style="text-align:center; margin:20px 0;">
 • • • •
</div>

# 1. **CONFIGURACIÓN DEL ENTORNO**

In [2]:
# Manipulación y análisis de datos
import pandas as pd
import numpy as np


# Visualización de datos
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
from sqlalchemy import create_engine
import sqlalchemy
import mysql.connector as mysql
from scipy.stats import ttest_ind, chi2_contingency
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.metrics import confusion_matrix
from scipy.stats import chi2_contingency, norm


# Sesgos
from scipy import stats

# Expresiones regulares
import re

#detección y corrección de errores ortográficos
from fuzzywuzzy import process

import warnings
warnings.filterwarnings('ignore')

# mi archivo con funciones definidas
import lib_propias as prop


# Nulos
import missingno as msno

# Fechas
from datetime import datetime


c:\Users\Azahara\anaconda3\envs\DA_TODO\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
# Configuración general de estilo para visualizaciones

# Tema visual clásico con fondo claro y grilla suave
plt.style.use('seaborn-v0_8')

# Paleta de colores moderna y profesional
sns.set_palette("husl")

# Tamaño estándar para gráficos (ancho x alto en pulgadas)
plt.rcParams['figure.figsize'] = (12, 6)

# Tamaño general de las fuentes en los gráficos
plt.rcParams['font.size'] = 11


In [4]:
import plotly.io as pio

# Crear un template personalizado similar al estilo seaborn-v0_8
pio.templates["custom"] = pio.templates["plotly_white"]

pio.templates["custom"].layout.update(
    font=dict(size=11, family='Arial'),
    width=1000,  # equivalente aprox a figsize=(12,6)
    height=500,
    plot_bgcolor='white',
    paper_bgcolor='white',
    xaxis=dict(showgrid=True, gridcolor='lightgrey'),
    yaxis=dict(showgrid=True, gridcolor='lightgrey'),
    colorway=px.colors.qualitative.Set2  # paleta similar a sns.husl
)

# Establecer este template como predeterminado
pio.templates.default = "custom"


<hr style="border:none; height:2px; background:linear-gradient(to right, #3498db, #e74c3c, #f39c12);">

<hr style="border:none; height:1px; background:#ddd;">

<div style="text-align:center; margin:20px 0;">
• • • •
</div>

# 2. **DESCARGA DE LOS DATOS PARA LA INVESTIACIÓN**

Usaremos el dataframe creado después de nuestra limpieza del original con las columnas traducidas al castellano. 

In [5]:
df_twoyears = pd.read_csv(r"C:\Users\Azahara\Documents\NUCLIO\PROYECTO FINAL\notebooks\compas_twoyears.csv")

<hr style="border:none; height:2px; background:linear-gradient(to right, #3498db, #e74c3c, #f39c12);">

<hr style="border:none; height:1px; background:#ddd;">

<div style="text-align:center; margin:20px 0;">
• • • •
</div>

# **3.PROBLEMÁTICAS BASICAS DEL DATASET DE PROPUBLICA TWOYEARS**

---

## **3.1 ENCONTRADAS DESDE NUESTRO PUNTO DE VISTA**

En el momento en que iniciamos el análisis estadístico básico observamos datos que nos parecieron incoherentes. Ejecutamos describe() y observamos los resultados para: 
- dias_entre_arresto_evaluacion
- dias_desde_compas
- dias_entre_arrestos

In [6]:
# Estadísticas numéricas básicas
df_twoyears.describe(include='number').T


,count,mean,std,min,25%,50%,75%,max
id_persona,7214.0,5501.255753,3175.706870,1.0,2735.25,5509.5,8246.5,11001.0
edad,7214.0,34.817993,11.888922,18.0,25.00,31.0,42.0,96.0
num_anteced_juv,7214.0,0.067230,0.473972,0.0,0.00,0.0,0.0,20.0
resultado_compas,7214.0,4.509565,2.856396,1.0,2.00,4.0,7.0,10.0
num_delitomenor_juv,7214.0,0.090934,0.485239,0.0,0.00,0.0,0.0,13.0
num_otrosdelitos_juv,7214.0,0.109371,0.501586,0.0,0.00,0.0,0.0,17.0
num_antecedentes_totales,7214.0,3.472415,4.882538,0.0,0.00,2.0,5.0,38.0
dias_entre_arresto_evaluacion,6907.0,3.304763,75.809505,-414.0,-1.00,-1.0,0.0,1057.0
dias_desde_compas,7192.0,57.731368,329.740215,0.0,1.00,1.0,2.0,9485.0
dias_entre_arrestos,2316.0,20.269430,74.871668,-1.0,0.00,0.0,1.0,993.0


1. **DIAS ENTRE EL ARRESTO Y LA EVALUACIÓN(-414 a 1057)**

- Lo lógico es que el test COMPAS se haga en los días cercanos al arresto (mismo día o días posteriores).
- Un valor negativo implica que el test se aplicó antes de que ocurriera el arresto, lo cual no tiene sentido → probablemente error de registro.
- Un valor tan alto como 1057 días (~3 años) indica que el test se aplicó años después del arresto, cuando ya no serviría para predecir el riesgo inmediato → otro error probable.

👉 Decisión : eliminar filas con valores negativos o muy grandes (ej. > 30 días). De hecho, ProPublica en su análisis eliminó casos donde la evaluación no se hizo en ±30 días respecto al arresto.

2. **DÍAS DESDE COMPAS HASTA EL EVENTO**(hasta 9485 ≈ 25 años)

- Esta variable debería reflejar el tiempo transcurrido tras la evaluación COMPAS.
- El dataset se recopiló en 2013–2014 y se siguió a los acusados hasta marzo de 2016 → el máximo esperado es unos 1200–1460 días (≈ 4 años).
- Valores de 25 años son imposibles, seguramente errores de cálculo o registros históricos que se colaron.

👉 Decisión: eliminar o recortar a un máximo razonable (ej. 730 días = 2 años).

3. **DIAS ENTRE ARRESTOS** (0, 1, hasta 993)

- Aquí se mide el intervalo entre un arresto y el siguiente.
- Ceros → mismo día (posible si se registran múltiples cargos en la misma fecha).
- Valores pequeños (1,2 días) → factibles (una persona es detenida de nuevo al poco tiempo).
- Valores grandes (ej. 993 ≈ 3 años) → siguen siendo posibles, pero hay que contrastar: si estás limitando el análisis a reincidencia en dos años, no deberían entrar intervalos tan largos.

👉 Decisión: Mantener los valores realistas (0 a 730 días, es decir hasta 2 años). Eliminar filas con valores mayores a 730, porque no cumplen con tu ventana de análisis.

## **3.2 DISCUTIDAS POR BARESTEIN**

El autor inicia el estudio de los datos de Propublica debido a que esos datos se utilizan en un número cada vez más creciente para poner a prueba la equidad algorítmica. Es así que, como nosotros, examina más de cerca los conjuntos de datos elaborados por estos periodistas de Propublica. 

Mediante un análisis de la distribución de los acusados a lo largo de las fechas de evaluación de Compas se percata de que hay un corte a partir del 1 de abril de 2016 tan solo para los no reincidentes, dejando en análisis aquellos que sí que reincidieron y creando la hipótesis de que el sesgo implícito en los datos va a favor de aumentar la tasa de reincidentes. 

De alguna manera el nombre asignado al dataset iba emparejado a la intencion de hacer un análisis a dos años, bien, tal vez habrá que renombrarlo a "year_and_threemonths". 

Barestein tenía acceso a unos datos que llamó completos y realiza una comparativa con varios gráficos para visualizar el número de personas que realizan compas semana a semana. Tiene dos gráficos por grupo, personas identificadas como reincidentes a dos años, y no reincidentes (página 7, [Barestein, 2019](https://arxiv.org/pdf/1906.04711))



Nosotros vamos a emular de manea algo diferente ese gráfico para demostrar visualmente dicho corte:

In [7]:
#CODIGO PARA LLEVAR AL OTRO NOTEBOOK DONDE ANALIZAR LA RECOLECCION DE DATOS DE PROPUBLICA PARA EL TWOYEARS SEGÚN BARSTEIN
# PARA VER EL CORTE A FECHA 30/03/16 PARA NO REINCIDENTES

def graficar_barras_semanales_px(df, fecha_col, categoria_col):
    """
    Gráfico de barras: número de personas evaluadas por semana
    separado por categoría (ej: reincide True/False).
    
    Parámetros:
        df : DataFrame
        fecha_col : str -> columna con las fechas (ej: 'fecha_compas')
        categoria_col : str -> columna con la categoría (ej: 'reincide')
    """
    # Asegurar formato datetime
    df[fecha_col] = pd.to_datetime(df[fecha_col], errors='coerce')

    # Crear columna semana
    df['semana'] = df[fecha_col].dt.to_period('W').apply(lambda r: r.start_time)

    # Contar personas por semana y categoría
    conteo = df.groupby(['semana', categoria_col]).size().reset_index(name='num_personas')

    # Gráfico de barras agrupadas
    fig = px.bar(
        conteo,
        x='semana',
        y='num_personas',
        color=categoria_col,
        barmode='group',
        text='num_personas',
        labels={'semana': 'Semana', 'num_personas': 'Número de personas'},
        title=f'Número de personas evaluadas por semana según {categoria_col}'
    )

    fig.update_traces(texttemplate='%{text}', textposition='outside')
    fig.update_layout(
        plot_bgcolor='white',
        xaxis_showgrid=True, xaxis_gridcolor='lightgrey',
        yaxis_showgrid=True, yaxis_gridcolor='lightgrey'
    )

    fig.show()

In [8]:
graficar_barras_semanales_px(df_twoyears, 'fecha_evaluacion_compas', 'reincide' )

**✅ QUEDA REFLEJADO EL CORTE DE DATOS EN CASOS SIN REINCIDENCIA A PARTIR DEL 1 DE ABRIL DE 2014, LO QUE ES SUFICIENTE PARA NOSOTROS PARA ACEPTAR LA TEORÍA DE SESGO EN LA INCLUSIÓN DE DATOS**

<hr style="border:none; height:2px; background:linear-gradient(to right, #3498db, #e74c3c, #f39c12);">

<hr style="border:none; height:1px; background:#ddd;">

<div style="text-align:center; margin:20px 0;">
• • • •
</div>

# **4. QUÉ DATOS DEBEN ESTAR EN EL DATASET PARA EVITAR SESGOS**

Modifico df_twoyears con motivos de inclusión de los casos  coherentes a:
- Días entre arresto y Compas: max 30 días. 
- Días de evento desde Compas: max dos años, 730 días.
- Días entre arrestos: max dos años, 730 días. 
- Corte Barestein.

Pero no voy a filtrarlo todo de golpe, si no que voy a crear 3 datasets filtro a filtro para valorar el volumen total del dataset y poder investigar sobre los datos resultantes y los que serían eliminados. 

A partir de ahora iremos analizando y comparando los datos que serían eliminados en comparativa a los que quedan. 

4. **Hipótesis de Filtrado y Evaluación**
   - **4.1 Días entre el arresto y la evaluación (-414 a 1057)**
     - Explicación del problema
     - Creación de `df_filtrado_eval` aplicando este filtro
     - Análisis básico del nuevo dataframe
   - **4.2 Días desde COMPAS hasta el evento (hasta 9485 ≈ 25 años)**
     - Explicación del problema
     - Creación de `df_filtrado_evento` aplicando este filtro
     - Análisis básico del nuevo dataframe
   - **4.3 Días entre arrestos (0 a 993)**
     - Explicación del problema
     - Creación de `df_filtrado_arrestos` aplicando este filtro
     - Análisis básico del nuevo dataframe
   - **4.4 Combinación de filtros**
     - Creación de `df_filtrado_total` aplicando todas las hipótesis
     - Comparación con el dataset original

---

## **4.1 DIAS ENTRE EL ARRESTO Y LA EVALUCACIÓN COMPAS**

**EXPLICACIÓN DEL PROBLEMA** 

¿Qué dijo ProPublica sobre el periodo de evaluación COMPAS respecto al arresto?

En su análisis original, ProPublica precisa que para asociar correctamente cada puntuación COMPAS con sus respectivos casos, solamente incluyeron los registros donde la fecha de arresto o de imputación ocurría dentro de los 30 días de la realización del test. En aquellos casos donde no pudieron establecer esta correspondencia, fueron excluidos del análisis 
ProPublica. Referencias (https://www.propublica.org/article/how-we-analyzed-the-compas-recidivism-algorithm)

Esto significa que, según ProPublica, solo se trabajó con evaluaciones hechas en una ventana razonable alrededor del arresto: ±30 días.

Poniendo todo junto: **esto es muy importante, los criterios de inclusión deben ser respetados**. 
ProPublica estableció explícitamente un criterio razonable de ventana temporal (±30 días entre arresto y evaluación) para asegurar congruencia en los datos.

  - Explicación del problema
     - Creación de `df_filtrado_eval` aplicando este filtro
     - Análisis básico del nuevo dataframe

**CREACIÓN DEL FILTRADO POR DIAS ENTRE ARRESTO Y EVALUACIÓN**

In [9]:
# Copia del df original
df_twoyears_clean_diasArrestoCompas = df_twoyears.copy()

# 1. Filtrar días entre arresto y evaluación (mantener ±30 días)
df_twoyears_clean_diasArrestoCompas = df_twoyears_clean_diasArrestoCompas[(df_twoyears_clean_diasArrestoCompas['dias_entre_arresto_evaluacion'] >= -30) & 
                    (df_twoyears_clean_diasArrestoCompas['dias_entre_arresto_evaluacion'] <= 30)]

# 2. Filtrar para tener df de los casos que quedan fuera.
df_twoyears_dirty_diasArrestoCompas = df_twoyears.loc[~df_twoyears['dias_entre_arresto_evaluacion'].between(-30, 30, inclusive='both') | df_twoyears['dias_entre_arresto_evaluacion'].isna()].copy()


# 3. Hacer comparativa de ambos datasets en resultados de reincidencia.

print(f"Dimensiones antes: {df_twoyears.shape}")
print(f"Dimensiones después: {df_twoyears_clean_diasArrestoCompas.shape}")
print (f"Dimensiones excluidos: {df_twoyears_dirty_diasArrestoCompas.shape}")
print(f"Se han extraído del análisis {len(df_twoyears) - len(df_twoyears_clean_diasArrestoCompas)} casos")


Dimensiones antes: (7214, 54)
Dimensiones después: (6172, 54)
Dimensiones excluidos: (1042, 54)
Se han extraído del análisis 1042 casos


Si somos literales con el margen de 0 a +30 días y no toleramos ninguna fecha en negativo de la evaluación Compas de golpe se pierden muchos datos:

In [10]:
# Copia del df original
df_twoyears_clean_diasArrestoCompas2 = df_twoyears.copy()

# 1. Filtrar días entre arresto y evaluación (mantener >=30 días)
df_twoyears_clean_diasArrestoCompas2 = df_twoyears_clean_diasArrestoCompas2[
    (df_twoyears_clean_diasArrestoCompas2['dias_entre_arresto_evaluacion'] >= 0) & 
    (df_twoyears_clean_diasArrestoCompas2['dias_entre_arresto_evaluacion'] <= 30)
]

# 2. Filtrar para tener df de los casos que quedan fuera.
df_twoyears_dirty_diasArrestoCompas2 = df_twoyears.loc[~df_twoyears['dias_entre_arresto_evaluacion'].between(0, 30, inclusive='both') | df_twoyears['dias_entre_arresto_evaluacion'].isna()].copy()


# 3. Hacer comparativa de ambos datasets en resultados de reincidencia.

print(f"Dimensiones antes: {df_twoyears.shape}")
print(f"Dimensiones después: {df_twoyears_clean_diasArrestoCompas2.shape}")
print (f"Dimensiones excluidos: {df_twoyears_dirty_diasArrestoCompas2.shape}")
print(f"Se han extraído del análisis {len(df_twoyears) - len(df_twoyears_clean_diasArrestoCompas2)} casos")


Dimensiones antes: (7214, 54)
Dimensiones después: (1448, 54)
Dimensiones excluidos: (5766, 54)
Se han extraído del análisis 5766 casos


Por este motivo mantenemos +-30 desde fecha de Compas intentando incluir a una muestra mayor. 

En este caso creamos también un dataframe con los datos excluídos para sacar de esos datos insights imporantes.

Vamos a comparar los datos que se quedan fuera comparados con los datos del dataset origianl Twoyears usando el filtrado -30 a +30 días de arresto a Compas. 

In [11]:
# Comparar los datos que estoy dejando fuera con los del dataset original:

# --- Configuración ---
SCORE_COL  = 'resultado_compas'   # columna resultado COMPAS
TARGET_COL = 'reincide'     # columna reincidencia
LABEL_A = 'CLEAN (±30 días)'
LABEL_B = 'DIRTY (fuera ±30 o NaN)'

df_a = df_twoyears_clean_diasArrestoCompas.copy()
df_b = df_twoyears_dirty_diasArrestoCompas.copy()

def comparar_columna_plotly(col, titulo):
    # Crear tabla con frecuencias relativas (%)
    s_a = df_a[col].value_counts(normalize=True).mul(100).round(1)
    s_b = df_b[col].value_counts(normalize=True).mul(100).round(1)
    tabla = pd.DataFrame({LABEL_A: s_a, LABEL_B: s_b}).fillna(0).reset_index().rename(columns={'index': col})
    
    # Gráfico de barras agrupadas
    fig = px.bar(tabla, x=col, y=[LABEL_A, LABEL_B], barmode='group', text_auto=True,
                 title=titulo)
    
    # Estilo solicitado
    fig.update_layout(
        xaxis_title=col,
        yaxis_title='Porcentaje (%)',
        plot_bgcolor='rgba(255,255,255,1)',
        xaxis_showgrid=True, xaxis_gridcolor='lightgrey',
        yaxis_showgrid=True, yaxis_gridcolor='lightgrey'
    )
    fig.show()
    return tabla

# --- Comparaciones ---
tabla_score = comparar_columna_plotly(SCORE_COL, "Distribución Resultado COMPAS — CLEAN vs DIRTY")
tabla_reinc = comparar_columna_plotly(TARGET_COL, "Distribución Reincidencia — CLEAN vs DIRTY")


### **CONCLUSIÓN SOBRE LA REINCIDENCIA** — Comparación CLEAN (±30 días) vs DIRTY (fuera de ±30 o NaN)

**Tendencia general**  
- Ambos grupos muestran proporciones similares, pero con matices:  
  - En el grupo **DIRTY** hay algo más de personas sin reincidencia.  
  - La reincidencia es ligeramente menor en el grupo **DIRTY** (42,4%) que en el **CLEAN** (45,5%).  

**Posible explicación**  
- Los registros del grupo dirty incluyen valores temporales incoherentes (ej. -414 días o +1057 días).  
- Estos errores de calidad de datos pueden sesgar la tasa de reincidencia hacia abajo, reduciendo artificialmente el porcentaje de reincidentes.  
- Incluirlos en el análisis puede distorsionar la estimación real del riesgo.  

**Decisión analítica**  
- Separar los casos dirty es la mejor opción, ya que no siguen un criterio lógico y pueden contaminar las conclusiones.  
- Para evaluar la calidad predictiva de COMPAS lo más sólido es trabajar con el grupo **CLEAN (±30 días)**.  
- Los casos **DIRTY** deben documentarse como inconsistencias en los datos, pero no mezclarse en el análisis principal.  

📌  
Los registros *dirty* presentan una tasa de reincidencia algo menor, lo que demuestra que además de ser un problema de calidad de datos, pueden sesgar las métricas de reincidencia. Por ello conviene excluirlos del análisis principal.


El grupo “dirty” presenta una tasa de reincidencia algo menor que el grupo “clean”, lo que indica que los registros con fechas incoherentes no solo son un problema de calidad de datos, sino que además pueden estar sesgando las métricas de reincidencia, por lo que conviene excluirlos del análisis principal.

---

## **4.2 DIAS DESDE COMPAS HASTA EL EVENTO (hasta 9485 días, 25 años)**

Al explorar esta variable se detectan **valores extremadamente altos** (hasta más de 9.000 días). Estos registros no son coherentes con el periodo de observación real del dataset, ya que exceden con mucho el **horizonte temporal de seguimiento** establecido para los acusados. 

La presencia de cifras tan alejadas de lo esperado indica problemas de calidad de datos, probablemente debidos a errores de cálculo o a la inclusión de eventos que no corresponden al marco del estudio.

Acción tomada: se opta por limitar esta variable a un rango máximo razonable de análisis (ej. 730 días = 2 años), de modo que se mantenga la consistencia con el diseño del seguimiento y se reduzca el impacto de valores espurios en los resultados.

In [12]:
# Copia del df original
df_twoyears_clean_diasdesdeEvento = df_twoyears.copy()

# 1. Filtrar días desde COMPAS (máximo 2 años = 730 días)
df_twoyears_clean_diasdesdeEvento = df_twoyears_clean_diasdesdeEvento[
    (df_twoyears_clean_diasdesdeEvento['dias_desde_compas'] >= 0) &
    (df_twoyears_clean_diasdesdeEvento['dias_desde_compas'] <= 730)
]

# 2. Filtrar para tener df de los casos que quedan fuera (>730 o NaN)
df_twoyears_dirty_diasdesdeEvento = df_twoyears.loc[
    (df_twoyears['dias_desde_compas'] > 730) | df_twoyears['dias_desde_compas'].isna()
].copy()

# 3. Hacer comparativa de ambos datasets en resultados de reincidencia
print(f"Dimensiones antes: {df_twoyears.shape}")
print(f"Dimensiones después (clean ≤730): {df_twoyears_clean_diasdesdeEvento.shape}")
print(f"Dimensiones excluidos (dirty >730 o NaN): {df_twoyears_dirty_diasdesdeEvento.shape}")
print(f"Se han extraído del análisis {len(df_twoyears) - len(df_twoyears_clean_diasdesdeEvento)} casos")


Dimensiones antes: (7214, 54)
Dimensiones después (clean ≤730): (7048, 54)
Dimensiones excluidos (dirty >730 o NaN): (166, 54)
Se han extraído del análisis 166 casos


Vamos a comparar los datos que se quedan fuera comparados con los datos del dataset origianl Twoyears. 

In [13]:

# --- Configuración ---
SCORE_COL  = 'resultado_compas'   # columna resultado COMPAS
TARGET_COL = 'reincide'           # columna reincidencia

LABEL_C = 'CLEAN (≤730 días)'
LABEL_D = 'DIRTY (>730 o NaN)'

df_c = df_twoyears_clean_diasdesdeEvento.copy()
df_d = df_twoyears_dirty_diasdesdeEvento.copy()

def comparar_columna_plotly_evento(col, titulo):
    # Crear tabla con frecuencias relativas (%)
    s_c = df_c[col].value_counts(normalize=True).mul(100).round(1)
    s_d = df_d[col].value_counts(normalize=True).mul(100).round(1)
    tabla = (
        pd.DataFrame({LABEL_C: s_c, LABEL_D: s_d})
        .fillna(0)
        .reset_index()
        .rename(columns={'index': col})
    )

    # Gráfico de barras agrupadas
    fig = px.bar(
        tabla, x=col, y=[LABEL_C, LABEL_D],
        barmode='group', text_auto=True, title=titulo
    )

    # Estilo solicitado
    fig.update_layout(
        xaxis_title=col,
        yaxis_title='Porcentaje (%)',
        plot_bgcolor='rgba(255,255,255,1)',
        xaxis_showgrid=True, xaxis_gridcolor='lightgrey',
        yaxis_showgrid=True, yaxis_gridcolor='lightgrey'
    )
    fig.show()
    return tabla

# --- Comparaciones ---
tabla_score_evento = comparar_columna_plotly_evento(SCORE_COL,  "Resultado COMPAS — CLEAN(≤730) vs DIRTY(>730/NaN)")
tabla_reinc_evento = comparar_columna_plotly_evento(TARGET_COL, "Reincidencia — CLEAN(≤730) vs DIRTY(>730/NaN)")


**CONCLUSIÓN DE LOS DATOS EXCLUIDOS**
### Reincidencia — Comparación CLEAN (≤730 días) vs DIRTY (>730 o NaN)

- En el grupo **CLEAN** (tiempos coherentes, ≤730 días), la reincidencia se mantiene equilibrada:  
  - **54,5%** no reinciden.  
  - **45,5%** reinciden.  

- En el grupo **DIRTY** (tiempos inconsistentes, >730 días o NaN), la distribución cambia notablemente:  
  - **74,7%** no reinciden.  
  - Solo **25,3%** reinciden.  

👉 **Interpretación:** Los registros con valores anómalos en la variable de días muestran una tasa de reincidencia mucho menor. Esto sugiere que dichos casos pueden estar distorsionando la estimación real del riesgo. Por ello, resulta adecuado excluirlos del análisis principal y mantenerlos solo como evidencia de problemas de calidad de datos.


---

## **4.3 DIAS ENTRE ARRESTOS**

In [14]:
# Copia del df original
df_twoyears_clean_diasentreArrestos = df_twoyears.copy()



# 3. Filtrar días entre arrestos (máximo 730 días = 2 años)
df_twoyears_clean_diasentreArrestos = df_twoyears_clean_diasentreArrestos[(df_twoyears_clean_diasentreArrestos['dias_entre_arrestos'] >= 0) & 
                    (df_twoyears_clean_diasentreArrestos['dias_entre_arrestos'] <= 730)]

print(f"Dimensiones antes: {df_twoyears.shape}")
print(f"Dimensiones después: {df_twoyears_clean_diasentreArrestos.shape}")

Dimensiones antes: (7214, 54)
Dimensiones después: (2305, 54)


### **CONCLUSIONES AL FILTRO DÍAS ENTRE ARRESTOS** 

Al aplicar un filtro de máximo 730 días (2 años) para la variable *dias_entre_arrestos*,
observamos que el dataset se reduce drásticamente: de **7.214 filas originales** a solo **2.305 filas**.
Esto supone perder cerca del **70% de los casos**.

Tras analizarlo, consideramos que aplicar este corte tendría varias implicaciones:

- **Reducción de muestra:** con tan pocos registros, el análisis pierde representatividad y robustez.
- **Posible sesgo:** si los valores fuera de rango no están distribuidos de manera aleatoria,
podríamos estar excluyendo grupos concretos (por ejemplo, según etnia o reincidencia),
introduciendo un sesgo adicional en el análisis.
- **Consistencia metodológica:** a diferencia de otros filtros aplicados (±30 días entre arresto y COMPAS,
y ≤730 días desde COMPAS hasta el evento), en este caso no existe un criterio claro en el diseño original
del dataset que justifique este límite.

Por estos motivos, **hemos decidido no aplicar este filtro como criterio de exclusión**.
En su lugar, mantendremos la variable como un **indicador auxiliar de calidad de datos**, analizando:

- qué porcentaje de casos supera los 730 días,  
- cómo se distribuyen estos valores según la reincidencia y los grupos demográficos.

De este modo preservamos el tamaño muestral para los análisis principales, a la vez que documentamos
las incoherencias de la variable como una limitación del dataset.


---

## **4.4 COMBINACIÓN DE FILTROS PROPUESTA PARA QUE SEA UN DATASET MÁS COHERENTE**

Tal y como Propublica realizó la variable diana del estudio, "two_year_recid" en la versión original, "reincide" en la nuestra tuvo que analizar los cargos antiguos y posteriores a la evaluación Compas para saber si había reincidido primero. Esa columna se llamó "is_recid", o "reincidencia_general" para nosotros, con la que filtró despues qué casos habían reincidido dentro de los dos años de la fecha de Compas y así creo la variable más importante. 

Eso está haciendo que haya un corte realista y coherente con el supuesto objetivo del estudio, **un estudio a dos años con datos a dos años para todos**. 

Pero nosotros nos preguntamos, ¿Cómo va a ser coherente con ese corte a dos años si hemos visto fechas de delito inicial de 1987 a 2011 en los datos? ¿Cuántos de esos delitos iniciales se han considerado NO REINCIDENTES? ¿Como estan afectando los datos antiguos al dataset? ¿De qué manera se han introducido si se supone que Compas se evalúa inicalmente después de la detención una vez detenidos por el delito? 

Pues bien, hay maneras diferentes de evaluar Compas, y eso lo podíamos ver en el dataset RawCompas. Existen 4 posibles situaciones donde se realiza la evaluación Compas. 
- Pretrial: División que maneja a personas arrestadas pero que aún no han sido juzgada, evaluación básica para decisiones de fianza.
- Probation: Supervisión en la comunidad como alternativa a la prisión, evaluación básica para supervisión postsentencia. 
- DRRD: División de Recuperación de Drogas, evaluación básica para programas de drogas, evalua la severidad de la adicción.
- Broadway County: Agencia General del Condado, es una evaluación básica administrativa. 

Existen pues diferentes tipos de Compas como podemos ver, desde uno administrativo a otro destinado a conocer el estado de voluntad de desintoxicación. Y por otro lado los que hemos estado viendo desde el los datos en Twoyears, son Pretrial y podemos saberlo tambien porque el tipo de escala valorado "Risk of Reicidivism" está dentro de las escalas "Risk and prescreen" de valoraciones rápidas. Con todo esto, es fácil deducir que si estamos incorporando datos de evaluaciones Compas desde el 1 enero de 2013 ¿como es posible que se realice un Risk of Reicidivism sobre un delito de 1987?

**PROPUESTA DE FILTRADO FINAL**

In [15]:

# Copia del df original
df_twoyears_clean = df_twoyears.copy()

# 1) Filtros temporales ya definidos
df_twoyears_clean = df_twoyears_clean[
    df_twoyears_clean['dias_entre_arresto_evaluacion'].between(-30, 30, inclusive="both") &
    df_twoyears_clean['dias_desde_compas'].between(0, 730, inclusive="both")
]

# 2) Filtro Barestein: excluir evaluaciones COMPAS posteriores al 01/04/2014
#    (conservar solo fechas <= 2014-04-01)

# Localizador simple de columna de fecha de evaluación COMPAS
def col(df, *names):
    cols_lower = {c.lower(): c for c in df.columns}
    for n in names:
        if n in df.columns:
            return n
        if n.lower() in cols_lower:
            return cols_lower[n.lower()]
    return None

COMPAS_DATE_COL = col(
    df_twoyears_clean,
    'compas_screening_date', 'screening_date',
    'fecha_compas', 'fecha_evaluacion_compas'
)
if COMPAS_DATE_COL is None:
    raise KeyError("No encuentro la columna de fecha de evaluación COMPAS (p. ej. 'compas_screening_date'/'screening_date').")

# Asegurar tipo datetime
df_twoyears_clean[COMPAS_DATE_COL] = pd.to_datetime(df_twoyears_clean[COMPAS_DATE_COL], errors='coerce')

# Aplicar corte (mantener <= 2014-04-01). NaT se mantienen (solo excluimos las > corte).
cutoff = pd.Timestamp('2014-04-01')
df_twoyears_clean = df_twoyears_clean[df_twoyears_clean[COMPAS_DATE_COL].isna() | (df_twoyears_clean[COMPAS_DATE_COL] <= cutoff)]

# Resumen
print(f"Dimensiones originales: {df_twoyears.shape}")
print(f"Dimensiones después de ambos filtros: {df_twoyears.shape[0]} → {len(df_twoyears[ df_twoyears['dias_entre_arresto_evaluacion'].between(-30, 30, inclusive='both') & df_twoyears['dias_desde_compas'].between(0, 730, inclusive='both') ])}")
print(f"Dimensiones tras añadir Barestein (≤ 2014-04-01): {df_twoyears_clean.shape}")


Dimensiones originales: (7214, 54)
Dimensiones después de ambos filtros: 7214 → 6122
Dimensiones tras añadir Barestein (≤ 2014-04-01): (5259, 54)


In [16]:

# --- Configuración ---
SCORE_COL  = 'resultado_compas'   # columna resultado COMPAS
TARGET_COL = 'reincide'           # columna reincidencia

LABEL_C = 'CLEAN'
LABEL_D = 'TWOYEARS'

df_c = df_twoyears_clean.copy()
df_d = df_twoyears.copy()

def comparar_columna_plotly_evento(col, titulo):
    # Crear tabla con frecuencias relativas (%)
    s_c = df_c[col].value_counts(normalize=True).mul(100).round(1)
    s_d = df_d[col].value_counts(normalize=True).mul(100).round(1)
    tabla = (
        pd.DataFrame({LABEL_C: s_c, LABEL_D: s_d})
        .fillna(0)
        .reset_index()
        .rename(columns={'index': col})
    )

    # Gráfico de barras agrupadas
    fig = px.bar(
        tabla, x=col, y=[LABEL_C, LABEL_D],
        barmode='group', text_auto=True, title=titulo
    )

    # Estilo solicitado
    fig.update_layout(
        xaxis_title=col,
        yaxis_title='Porcentaje (%)',
        plot_bgcolor='rgba(255,255,255,1)',
        xaxis_showgrid=True, xaxis_gridcolor='lightgrey',
        yaxis_showgrid=True, yaxis_gridcolor='lightgrey'
    )
    fig.show()
    return tabla

# --- Comparaciones ---
tabla_score_evento = comparar_columna_plotly_evento(SCORE_COL,  "Resultado COMPAS — CLEAN vs TWOYEARS")
tabla_reinc_evento = comparar_columna_plotly_evento(TARGET_COL, "Reincidencia — CLEAN vs TWOYEARS")


### Comparativa de tasas de reincidencia — ProPublica, Barstein y nuestro filtrado

| Dataset                              | Total casos | % No reincide | % Reincide |
|--------------------------------------|-------------|---------------|------------|
| ProPublica (2016, two-year data)     | 7,214       | 54.9%         | **45.1%**  |
| Barstein (2019, corrected dataset)   | 6,216       | 63.8%         | **36.2%**  |
| Nuestro filtrado (±30, ≤730, ≤1/4/14)| 5,259       | 63.2%         | **36.8%**  |

- Diferencia absoluta ProPublica vs filtrado: **8.3 p.p.**  
- Diferencia relativa: la tasa de ProPublica es **22.6% más alta** que la real corregida.

👉 Con esto confirmamos que el error de incluir reincidentes posteriores a abril de 2014 infló artificialmente
la tasa de reincidencia a dos años en el análisis original de ProPublica.


<hr style="border:none; height:2px; background:linear-gradient(to right, #3498db, #e74c3c, #f39c12);">

<hr style="border:none; height:1px; background:#ddd;">

<div style="text-align:center; margin:20px 0;">
- • • • •
</div>

# **5. COMPARATIVA DATOS FILTRADOS VS PROPUBLICA**

---

## **5.1 SPEARMAN, RIESGO VS REINCIDENCIAPOR ETNIA**



Objetivo. Queremos comprobar si la relación entre la **puntuación COMPAS** (`resultado_compas`) y la **reincidencia real** (`reincide`) es parecida en los distintos grupos étnicos.

Método. Como estamos empezando y no asumimos normalidad, usamos **Spearman**, que mide la relación **monótona** entre dos variables. Calculamos Spearman por **etnia** en ambos datasets (ProPublica y Clean).


In [17]:

def _agg_riesgo_reincid(df, group_var):
    # Riesgo medio (1–10) -> escalar a 0–1
    riesgo = (
        df.groupby(group_var)['resultado_compas']
          .mean()
          .reset_index()
          .rename(columns={'resultado_compas':'riesgo_medio'})
    )
    riesgo['riesgo_medio_scaled'] = riesgo['riesgo_medio'] / 10.0

    # Reincidencia real (0–1)
    real = (
        df.groupby(group_var)['reincide']
          .mean()
          .reset_index()
          .rename(columns={'reincide':'reincidencia_real'})
    )
    return riesgo.merge(real, on=group_var)

def compare_risk_vs_recid_two(df_a, df_b, group_var, label_a='CLEAN', label_b='DIRTY'):
    """
    Compara riesgo medio (escalado) y reincidencia real por grupo entre dos datasets.
    Muestra barras paralelas por dataset y separa por métrica.
    Devuelve la tabla combinada.
    """
    comp_a = _agg_riesgo_reincid(df_a, group_var)
    comp_b = _agg_riesgo_reincid(df_b, group_var)

    comp_a['dataset'] = label_a
    comp_b['dataset'] = label_b

    comp = pd.concat([comp_a, comp_b], ignore_index=True)

    # Long para plot
    df_long = comp.melt(
        id_vars=[group_var,'dataset'],
        value_vars=['riesgo_medio_scaled','reincidencia_real'],
        var_name='metric', value_name='value'
    ).replace({'metric':{
        'riesgo_medio_scaled':'Riesgo medio (escalado)',
        'reincidencia_real':'Reincidencia real'
    }})

    # Barras paralelas (agrupadas) y facet por métrica
    fig = px.bar(
        df_long, x=group_var, y='value', color='dataset',
        barmode='group', text='value', facet_col='metric',
        title=f'Riesgo vs Reincidencia por {group_var} — {label_a} vs {label_b}'
    )
    fig.update_traces(texttemplate='%{text:.1%}', textposition='outside')
    fig.update_layout(
        yaxis_tickformat=',.0%', yaxis_title='Porcentaje',
        plot_bgcolor='rgba(255,255,255,1)',
        xaxis_showgrid=True, xaxis_gridcolor='lightgrey',
        yaxis_showgrid=True, yaxis_gridcolor='lightgrey'
    )
    fig.show()

    # Correlación de rangos (Spearman) en cada dataset (promedios por grupo)
    try:
        from scipy.stats import spearmanr
        rho_a, p_a = spearmanr(comp_a['riesgo_medio_scaled'], comp_a['reincidencia_real'])
        rho_b, p_b = spearmanr(comp_b['riesgo_medio_scaled'], comp_b['reincidencia_real'])
        print(f"[{label_a}] Spearman: rho={rho_a:.3f}, p={p_a:.4f}")
        print(f"[{label_b}] Spearman: rho={rho_b:.3f}, p={p_b:.4f}")
    except Exception as e:
        print(f"[Aviso] No se pudo calcular Spearman: {e}")

    return comp

# Ejecuto código
for g in [c for c in ['etnia','genero','rango_edad'] if c in df_twoyears.columns]:
     compare_risk_vs_recid_two(df_twoyears_clean, df_twoyears, g, 'CLEAN', 'PROPUBLICA')


[CLEAN] Spearman: rho=0.886, p=0.0188
[PROPUBLICA] Spearman: rho=1.000, p=0.0000


[CLEAN] Spearman: rho=1.000, p=nan
[PROPUBLICA] Spearman: rho=1.000, p=nan


[CLEAN] Spearman: rho=1.000, p=0.0000
[PROPUBLICA] Spearman: rho=1.000, p=0.0000


### Correlación de Spearman — Riesgo COMPAS vs Reincidencia real

Hemos evaluado la correlación de rangos (Spearman) entre el **riesgo medio escalado**
y la **tasa de reincidencia real**, comparando el dataset *CLEAN* (con filtros aplicados)
y el dataset original de *ProPublica*.

#### Resultados por etnia
- **CLEAN:** rho = 0.886, p = 0.0188  
  - Correlación **fuerte y positiva**, estadísticamente significativa.  
  - A mayor riesgo medio por etnia, mayor reincidencia real, aunque no perfecta.  
- **PROPUBLICA:** rho = 1.000, p = 0.0000  
  - Correlación **perfecta**, irreal en datos sociales.  
  - Explicada por el sesgo de ProPublica al incluir recidivistas posteriores al 1/04/2014.

#### Resultados por rango de edad
- **CLEAN:** rho = 1.000, p = 0.0000  
- **PROPUBLICA:** rho = 1.000, p = 0.0000  
  - En ambos casos aparece correlación **perfecta**.  
  - Esto refleja que los grupos de edad mantienen exactamente el mismo orden en riesgo y reincidencia (menores de 25 con mayor riesgo y reincidencia, mayores de 45 con menor).  
  - Aunque el valor es perfecto, aquí sí es plausible porque hay solo 3 grupos de edad claramente ordenados.

#### Resultados por género
- **CLEAN:** rho = 1.000, p = nan  
- **PROPUBLICA:** rho = 1.000, p = nan  
  - Con solo **dos grupos (hombre y mujer)**, Spearman devuelve una correlación perfecta por definición.  
  - El p-value aparece como *nan* porque no se puede calcular significación estadística con solo 2 puntos.



📌 **Conclusión general:**
- Con el dataset *CLEAN*, las correlaciones siguen siendo altas pero más realistas (ej. etnia, rho≈0.886).  
- En el dataset *ProPublica*, algunas correlaciones aparecen artificialmente perfectas debido al error de muestreo (ej. etnia, rho=1).  
- En variables con pocos grupos (género = 2, edad = 3), Spearman tiende a devolver correlaciones extremas (1.0), lo cual debe interpretarse con cuidado: más que una "predicción perfecta", refleja que los grupos están perfectamente ordenados de forma monótona.


---


## **5.2 DIFERENCIA GLOBAL DE TASAS: T-TEST DE WELCH Y χ² (uso descriptivo)**

En 5.2.1 hacemos la comparación descriptiva (Welch y χ²). En 5.2.2 realizamos el contraste formal con Incluidos vs Excluidos (χ² y test de dos proporciones con IC95%).

### **5.2.1 DIFERENCIA GLOBAL DE TASAS: T-TEST DE WELCH Y χ² (uso descriptivo)**



Objetivo. Comparar la **tasa global de reincidencia** entre ProPublica y nuestro dataset Clean.

Qué hacemos.  
- **t-test (Welch)** para comparar medias de `reincide` (equivale a comparar proporciones).  
- **χ²** en una tabla de contingencia (*dataset* × *reincide*).

⚠️ Nota importante sobre el supuesto de independencia.  
`df_twoyears_clean` es un **subconjunto** de `df_twoyears` (hay **solapamiento de filas**), por lo que **estos tests no cumplen** el supuesto de **muestras independientes**.  
> **Decisión**: usamos estos tests **solo como resumen descriptivo** para confirmar que la diferencia observada no es casual.


In [18]:


# --- 0) (Re)construir 'rates' desde los dos datasets ---
def build_rates(df_clean, df_propu,
                decil_col='resultado_compas', y_col='reincide'):
    rate_clean = (df_clean.groupby(decil_col, as_index=False)[y_col]
                  .mean().rename(columns={y_col: 'recid_clean'}))
    rate_propu = (df_propu.groupby(decil_col, as_index=False)[y_col]
                  .mean().rename(columns={y_col: 'recid_propu'}))
    rates = (rate_clean.merge(rate_propu, on=decil_col, how='inner')
                        .sort_values(decil_col))
    rates['diff'] = rates['recid_clean'] - rates['recid_propu']
    rates['diff_pp'] = rates['diff'] * 100
    return rates

rates = build_rates(df_twoyears_clean, df_twoyears)

# --- 1) Subplots: líneas (arriba) + barras (abajo) ---
fig = make_subplots(
    rows=2, cols=1, shared_xaxes=True,
    vertical_spacing=0.12,
    row_heights=[0.6, 0.4],
    subplot_titles=("Tasa de reincidencia por decil COMPAS",
                    "Diferencia de tasas (Clean – ProPublica) por decil")
)

# (1) Líneas
fig.add_trace(
    go.Scatter(
        x=rates['resultado_compas'], y=rates['recid_propu'],
        mode='lines+markers', name='ProPublica',
        line=dict(width=2), marker=dict(symbol='circle', size=8),
        hovertemplate='Decil %{x}<br>Tasa %{y:.2%}<extra></extra>'
    ),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(
        x=rates['resultado_compas'], y=rates['recid_clean'],
        mode='lines+markers', name='Clean',
        line=dict(width=2), marker=dict(symbol='square', size=8),
        hovertemplate='Decil %{x}<br>Tasa %{y:.2%}<extra></extra>'
    ),
    row=1, col=1
)

# Medias globales
mean_propu = df_twoyears['reincide'].mean()
mean_clean = df_twoyears_clean['reincide'].mean()
fig.add_hline(y=mean_propu, line_dash="dash", opacity=0.5,
              annotation_text=f"Media ProPublica {mean_propu:.2%}",
              row=1, col=1)
fig.add_hline(y=mean_clean, line_dash="dash", opacity=0.5,
              annotation_text=f"Media Clean {mean_clean:.2%}",
              row=1, col=1)

# (2) Barras de diferencia (p.p.)
fig.add_trace(
    go.Bar(
        x=rates['resultado_compas'], y=rates['diff_pp'],
        name='Clean – ProPublica (p.p.)',
        text=rates['diff_pp'].round(1), textposition='outside',
        hovertemplate='Decil %{x}<br>Diferencia %{y:.1f} p.p.<extra></extra>'
    ),
    row=2, col=1
)
fig.add_hline(y=0, line_dash="dash", opacity=0.5, row=2, col=1)

# Estilo homogéneo
fig.update_layout(
    plot_bgcolor='rgba(255,255,255,1)',
    xaxis_showgrid=True, xaxis_gridcolor='lightgrey',
    xaxis2_showgrid=True, xaxis2_gridcolor='lightgrey',
    yaxis_showgrid=True, yaxis_gridcolor='lightgrey',
    yaxis2_showgrid=True, yaxis2_gridcolor='lightgrey',
    legend_title_text='Dataset',
    margin=dict(l=70, r=40, t=80, b=60)
)
fig.update_xaxes(title_text="Decil COMPAS", dtick=1, row=2, col=1)
fig.update_yaxes(title_text="Tasa de reincidencia", tickformat=".0%", row=1, col=1)
fig.update_yaxes(title_text="Diferencia (p.p.)", row=2, col=1)

fig.show()


**Lectura de la figura**

- En **todos los deciles** la tasa de reincidencia de **ProPublica** está **por encima** de la de **Clean**.  
- La **media** también refleja esa diferencia: ProPublica ≈ **45,1%** frente a Clean ≈ **38,8%**.
- El patrón por decil es **creciente** (a mayor decil, mayor tasa) y las dos curvas son **muy parecidas** en forma; la diferencia es sobre todo de **nivel**.
- La banda inferior (barras) muestra **Clean − ProPublica** en **puntos porcentuales**: es **negativa en todos los deciles** (entre **−3,7 p.p.** y **−10 p.p.**, con el **máximo hueco** en deciles **5–6**).
- **Interpretación sencilla:** al limpiar el dataset hemos eliminado casos con más probabilidad de reincidencia; por eso, **en todos los tramos de riesgo** la tasa de Clean es **menor** que la de ProPublica.

**Implicación para el resto del análisis**

- Cuando comparemos métricas (matriz de confusión, FPR/TPR, etc.), debemos usar el **mismo umbral** en ambos datasets y tener en cuenta que **Clean parte de una base más baja** de reincidencia.


---

### **5.2.2 CONTRASTE FORMAL (Incluidos vs Excluidos dentro de ProPublica: χ² + test de dos proporciones con IC95%)**

**Objetivo.** Ver si la **limpieza de datos** cambia **de forma significativa** la tasa de reincidencia.

**En qué nos basamos.** Como nuestro dataset *Clean* es un **subconjunto** del original (*ProPublica*), no podemos tratar “Clean vs ProPublica” como **muestras independientes**. Por eso, para un contraste correcto comparamos **dos grupos disjuntos** **dentro** del dataset original:
- **Incluidos**: personas que permanecen en *Clean*.
- **Excluidos**: personas que fueron eliminadas por los filtros.

**Qué hacemos (resumen operativo).**
1. Creamos `df_casosexcluidos` como un **anti-join** de `df_twoyears` contra `df_twoyears_clean` (por **índice** o por una **clave** estable).
2. Calculamos la **tasa de reincidencia** en **Incluidos** y en **Excluidos**.
3. Aplicamos dos contrastes:
   - **χ² de independencia** sobre la tabla 2×2 (*Incluido/Excluido* × *Reincide Sí/No*).
   - **Test de dos proporciones** con **IC95%** para la diferencia absoluta **Δ = p_incl − p_excl**.

**Cómo lo leemos.** Si **p ≤ 0,05** y el **IC95% de Δ** **no** incluye 0, concluimos que la limpieza **cambia** la tasa de reincidencia.  
- Si **Δ < 0**, la tasa es **menor** en *Incluidos* (esperable si al limpiar quitamos muchos casos con alta probabilidad de reincidencia, como sugiere Barstein).  
Mostramos también un **gráfico de barras con IC** para visualizar ambas tasas.


1º CREAMOS DF DE CASOS EXCLUIDOS:

In [19]:
# --- Opción A (recomendada): por ÍNDICE (si df_twoyears_clean conservó el índice original) ---
df_casosexcluidos = df_twoyears.loc[~df_twoyears.index.isin(df_twoyears_clean.index)].copy()

print (f"Número de casos excluidos", df_casosexcluidos.shape)

Número de casos excluidos (1955, 54)


2º CREAMOS CÓDIGO PARA TASAS DE REINCIDENCIA, CHI-CUADRADO, TEST DE DOS PROPORCIONES CON IC95%.

In [20]:
# Alineamos NaN en 'reincide' del mismo modo en que limpiaste en el punto 4
y_clean = df_twoyears_clean['reincide'].dropna().astype(int)
y_exclu = df_casosexcluidos['reincide'].dropna().astype(int)

# 2x2 (Excluido/Incluido × Reincide Sí/No)
ct = pd.DataFrame({
    'No': [int((1 - y_exclu).sum()), int((1 - y_clean).sum())],
    'Sí': [int(y_exclu.sum()),        int(y_clean.sum())]
}, index=['Excluido','Incluido'])
print("Tabla de contingencia (Incluido/Excluido × Reincide):\n", ct, "\n")

# Chi-cuadrado
chi2, p_chi, dof, expected = chi2_contingency(ct)
print(f"Chi-cuadrado: chi2={chi2:.3f}, gl={dof}, p={p_chi:.6f}")

# Dos proporciones (Incluidos vs Excluidos)
X_incl, N_incl = int(y_clean.sum()),  int(y_clean.size)
X_excl, N_excl = int(y_exclu.sum()),  int(y_exclu.size)

p1 = X_incl / N_incl   # tasa incluidos
p2 = X_excl / N_excl   # tasa excluidos
diff = p1 - p2

# z con proporción combinada
p_pool = (X_incl + X_excl) / (N_incl + N_excl)
se_pool = np.sqrt(p_pool*(1-p_pool)*(1/N_incl + 1/N_excl))
z  = diff / se_pool
p_z = 2*(1 - norm.cdf(abs(z)))

# IC95% para la diferencia (aprox. normal no combinada)
se_diff = np.sqrt(p1*(1-p1)/N_incl + p2*(1-p2)/N_excl)
lo = diff - 1.96*se_diff
hi = diff + 1.96*se_diff

print(f"Dos proporciones: p_incl={p1:.4f}, p_excl={p2:.4f}, Δ={diff:.4f}, IC95%=[{lo:.4f}, {hi:.4f}], z={z:.3f}, p={p_z:.6f}")

# Visual simple con IC
rates = pd.DataFrame({
    'Grupo': ['Incluido','Excluido'],
    'Tasa':  [p1, p2],
    'N':     [N_incl, N_excl]
})
rates['se'] = np.sqrt(rates['Tasa']*(1-rates['Tasa'])/rates['N'])
rates['lo'] = rates['Tasa'] - 1.96*rates['se']
rates['hi'] = rates['Tasa'] + 1.96*rates['se']

fig = px.bar(
    rates, x='Grupo', y='Tasa',
    text=rates['Tasa'].map(lambda v: f"{v*100:.1f}%"),
    error_y=rates['hi']-rates['Tasa'],
    error_y_minus=rates['Tasa']-rates['lo'],
    title="Tasa de reincidencia: Incluidos vs Excluidos (con IC95%)"
)
fig.update_traces(textposition='outside')
fig.update_layout(
    plot_bgcolor='white',
    xaxis_showgrid=True, xaxis_gridcolor='lightgrey',
    yaxis_showgrid=True, yaxis_gridcolor='lightgrey',
    yaxis_tickformat=".0%"
)
fig.show()

# Texto listo para informe
md = f"""
**Comparación formal (Incluidos vs Excluidos).** χ²={chi2:.2f}, gl={dof}, p={p_chi:.3g}.
Tasa en *Incluidos* = {p1*100:.1f}%, en *Excluidos* = {p2*100:.1f}%.
Δ = {diff*100:.1f} p.p. (IC95% [{lo*100:.1f}, {hi*100:.1f}] p.p.), z={z:.2f}, p={p_z:.3g}.
"""
print(md)


Tabla de contingencia (Incluido/Excluido × Reincide):
             No    Sí
Excluido   639  1316
Incluido  3324  1935 

Chi-cuadrado: chi2=535.020, gl=1, p=0.000000
Dos proporciones: p_incl=0.3679, p_excl=0.6731, Δ=-0.3052, IC95%=[-0.3297, -0.2807], z=-23.157, p=0.000000



**Comparación formal (Incluidos vs Excluidos).** χ²=535.02, gl=1, p=2.29e-118.
Tasa en *Incluidos* = 36.8%, en *Excluidos* = 67.3%.
Δ = -30.5 p.p. (IC95% [-33.0, -28.1] p.p.), z=-23.16, p=0.



**Resultados (Incluidos vs Excluidos)**  
- **Tamaños:** Incluidos **N=5.259**, Excluidos **N=1.955**.  
- **Tasas de reincidencia:**  
  - Incluidos: **36,8%**  
  - Excluidos: **67,3%**  
  - **Δ = −30,5 p.p.** (Incluidos − Excluidos)  
  - **IC95% de Δ:** [−33,0, −28,1] p.p.
- **Contrastes:**  
  - **Chi-cuadrado:** χ² = **535,02**, gl = 1, **p < 0,001**  
  - **Dos proporciones (z):** z = **−23,16**, **p < 0,001**

**Lectura sencilla.** Aproximadamente **2 de cada 3** personas en *Excluidos* reinciden (67%), mientras que en *Incluidos* es **1 de cada 3** (37%). La diferencia es **muy grande** y **estadísticamente significativa**. Esto confirma que la **limpieza** eliminó sobre todo casos con mayor probabilidad de reincidencia, y por eso la **media global de ProPublica** es **más alta** que la de nuestro **dataset limpio**.

**Implicación para el informe.**  
- Este resultado **justifica** la creación del dataset limpio: reduce sesgos de medición/registro y cambia de forma sustancial la tasa observada.  
- A partir de aquí, cuando comparemos métricas (matrices de confusión, FPR/TPR, etc.), debemos **dejar claro** con qué dataset trabajamos y mantener **el mismo umbral**.


---

## **5.3 MATRIZ DE CONFUSIÓN / TABLAS DE VERDAD**



Objetivo. Evaluar cómo clasificaría COMPAS en **alto riesgo** (positivo) usando el corte estándar **≥5** (Medium+High) frente a **<5** (Low).

Qué mostramos.  
- **Matrices de confusión** (heatmaps) para ProPublica y Clean.  
- **Métricas clave**: Accuracy, **TPR** (recall), **FPR** (tasa de falsos positivos), **TNR** (especificidad), Precision, F1 y **Balanced Accuracy**.  
- Visualizamos una **tabla comparativa**, **heatmaps** y un **gráfico de barras**.
- **Sensibilidad** con corte **≥8** (High puro) para ver el intercambio FP↔FN.


Usar matriz de confusión y sus métricas (FPR, TPR, especificidad, etc.) Necesitamos convertir el score en una predicción binaria aplicando un umbral al decil COMPAS.

Northopint decidió que los valores Low/Medium/High se dividieran así: 1-4, 5-7, 8-10. Pero para binarizar Propublica y Barestein decidieron el colapso 1–4 vs 5–10 (≥5) que vamos a mantener. Para binario, agrupan Medium+High como “High” (positivo), y Low como “Low” (negativo).

**DECISION**: Regla de binarización documentada: para binarizar Propublica y Barestein decidieron el colapso 1–4 vs 5–10 (≥5) que vamos a mantener. 

> Nota: aquí comparamos de forma **descriptiva** al mismo umbral. (Las pruebas formales ya las hicimos en el 5.2.)

<div align="center">
<h2>Matriz de confusión y métricas (umbral ≥ 5)</h2>

In [21]:
# --- Estilo homogéneo visualizacion gráficos ---
def aplicar_estilo(fig, x_title=None, y_title=None, legend_title=''):
    fig.update_layout(
        plot_bgcolor='rgba(255, 255, 255, 1)',
        xaxis_showgrid=True, xaxis_gridcolor='lightgrey',
        yaxis_showgrid=True, yaxis_gridcolor='lightgrey',
        legend_title_text=legend_title,
        margin=dict(l=70, r=40, t=60, b=60)
    )
    if x_title: fig.update_xaxes(title_text=x_title)
    if y_title: fig.update_yaxes(title_text=y_title)
    return fig

# --- Métricas a partir de una matriz de confusión ---
def metrics_from_cm(tp, fp, fn, tn):
    acc = (tp+tn)/max(tp+fp+fn+tn, 1)
    tpr = tp/max(tp+fn, 1)  # recall / sensibilidad
    fpr = fp/max(fp+tn, 1)
    tnr = tn/max(tn+fp, 1)  # especificidad
    ppv = tp/max(tp+fp, 1)  # precision
    npv = tn/max(tn+fn, 1)
    f1  = 2*ppv*tpr/max(ppv+tpr, 1e-12)
    bal = (tpr+tnr)/2
    return dict(Accuracy=acc, TPR=tpr, FPR=fpr, TNR=tnr, Precision=ppv, NPV=npv, F1=f1, BalancedAcc=bal)

# --- Construir predicción binaria a partir del decil ---
def get_cm_and_metrics(df, threshold=5, y_true='reincide', score='resultado_compas', label=''):
    y = df[y_true].astype(int).values
    yhat = (df[score] >= threshold).astype(int).values
    tn, fp, fn, tp = confusion_matrix(y, yhat, labels=[0,1]).ravel()
    m = metrics_from_cm(tp, fp, fn, tn)
    cm_counts = np.array([[tp, fn],
                          [fp, tn]])  # formato [fila: Real Sí/No] x [col: Pred Sí/No] para mostrar claro
    support = dict(Pos=tp+fn, Neg=tn+fp, N=len(y))
    return {'label': label, 'tp': tp, 'fp': fp, 'fn': fn, 'tn': tn, 'metrics': m, 'cm': cm_counts, 'support': support}

# === 1) Calcular métricas y CMs para ambos datasets (umbral ≥5) ===
res_propu = get_cm_and_metrics(df_twoyears,       threshold=5, label='ProPublica')
res_clean = get_cm_and_metrics(df_twoyears_clean, threshold=5, label='Clean')

# === 2A) TABLA-RESUMEN DE MÉTRICAS (profesional) ===
def format_pct(x): 
    return f"{100*x:.1f}%" if pd.notnull(x) else "—"

summary = pd.DataFrame({
    'Métrica': ['Accuracy', 'TPR (Recall)', 'FPR', 'TNR (Especificidad)', 'Precision (PPV)', 'F1', 'BalancedAcc'],
    'ProPublica': [res_propu['metrics'][k] for k in ['Accuracy','TPR','FPR','TNR','Precision','F1','BalancedAcc']],
    'Clean':      [res_clean['metrics'][k] for k in ['Accuracy','TPR','FPR','TNR','Precision','F1','BalancedAcc']],
})
summary['Δ (Clean − ProPublica)'] = summary['Clean'] - summary['ProPublica']

# Formateo a texto para Table
summary_fmt = summary.copy()
for col in ['ProPublica','Clean','Δ (Clean − ProPublica)']:
    summary_fmt[col] = summary_fmt[col].map(format_pct)

table_fig = go.Figure(data=[go.Table(
    header=dict(values=list(summary_fmt.columns), fill_color="#fc9278", align='left'),
    cells=dict(values=[summary_fmt[c] for c in summary_fmt.columns], align='left')
)])
table_fig.update_layout(title="Resumen de métricas a umbral ≥5")
table_fig.show()

# === 2B) HEATMAPS DE MATRIZ DE CONFUSIÓN (lado a lado) ===
# Texto por celda: cuenta y porcentaje fila
def cm_text(cm):
    row_sums = cm.sum(axis=1, keepdims=True)
    pct = np.divide(cm, np.where(row_sums==0, 1, row_sums), where=row_sums!=0)
    return np.array([[f"{cm[0,0]}\n({pct[0,0]*100:.1f}%)", f"{cm[0,1]}\n({pct[0,1]*100:.1f}%)"],
                     [f"{cm[1,0]}\n({pct[1,0]*100:.1f}%)", f"{cm[1,1]}\n({pct[1,1]*100:.1f}%)"]])

labels_x = ['Pred: Sí', 'Pred: No']
labels_y = ['Real: Sí', 'Real: No']

fig_cm = make_subplots(rows=1, cols=2, subplot_titles=(f"Matriz - ProPublica (N={res_propu['support']['N']})",
                                                       f"Matriz - Clean (N={res_clean['support']['N']})"))

fig_cm.add_trace(go.Heatmap(
    z=res_propu['cm'], x=labels_x, y=labels_y, colorscale="GnBu", showscale=False,
    text=cm_text(res_propu['cm']), texttemplate="%{text}", hovertemplate="%{y} / %{x}<br>Casos: %{z}<extra></extra>"
), row=1, col=1)

fig_cm.add_trace(go.Heatmap(
    z=res_clean['cm'], x=labels_x, y=labels_y, colorscale='GnBu', showscale=False,
    text=cm_text(res_clean['cm']), texttemplate="%{text}", hovertemplate="%{y} / %{x}<br>Casos: %{z}<extra></extra>"
), row=1, col=2)

aplicar_estilo(fig_cm, legend_title='')
fig_cm.update_xaxes(showgrid=False)
fig_cm.update_yaxes(showgrid=False)
fig_cm.update_layout(title="Matrices de confusión a umbral ≥5")
fig_cm.show()

# === 2C) BARRAS COMPARATIVAS DE MÉTRICAS CLAVE ===
plot_metrics = ['Accuracy', 'TPR', 'FPR', 'TNR', 'BalancedAcc']
tidy = (pd.DataFrame({
    'metric': plot_metrics,
    'ProPublica': [res_propu['metrics'][m] for m in plot_metrics],
    'Clean':      [res_clean['metrics'][m] for m in plot_metrics]
})
.melt(id_vars='metric', var_name='dataset', value_name='value'))

fig_bar = px.bar(tidy, x='metric', y='value', color='dataset', barmode='group',
                 title='Comparativa de métricas a umbral ≥5', text=tidy['value'].map(lambda v: f"{v*100:.1f}%"))
fig_bar.update_traces(textposition='outside')
fig_bar.update_yaxes(tickformat=".0%")
fig_bar = aplicar_estilo(fig_bar, x_title="Métrica", y_title="Valor")
fig_bar.show()




**Lectura de la tabla y las matrices**

- **Accuracy**: sube de **65,4%** a **67,0%** en *Clean* (**+1,6 p.p.**).  
- **FPR (falsos positivos)**: baja de **32,3%** a **30,0%** (**−2,4 p.p.**).  
  > Traducción: por cada **100 no reincidentes**, en *Clean* etiquetamos como “alto riesgo” a **2–3 personas menos**.
- **TNR (especificidad)**: sube de **67,7%** a **70,0%** (**+2,4 p.p.**).  
- **TPR (recall)**: cae ligeramente de **62,8%** a **61,8%** (**−0,8 p.p.**).  
  > Captamos casi lo mismo de quienes sí reinciden, con una pérdida pequeña.
- **Precision (PPV)**: baja de **61,4%** a **54,5%** (**−6,8 p.p.**).  
  > Entre los etiquetados como “alto riesgo”, la proporción que realmente reincide es menor. Es coherente con que la **prevalencia** de reincidencia sea más baja en *Clean*.
- **F1**: baja de **62,0%** a **57,9%** (**−4,1 p.p.**).  
- **Balanced Accuracy** (media de TPR y TNR): mejora de **65,1%** a **65,9%** (**+0,8 p.p.**).

**Qué nos dice esto**

- La **limpieza** logra un clasificador **más conservador con los no reincidentes**: **menos falsos positivos** y **más especificidad**, a costa de una **leve caída** del recall y del F1.  
- El **equilibrio global** mejora un poco (**BalancedAcc +0,8 p.p.**), y la **accuracy** también (**+1,6 p.p.**).

**Mensaje para decisión**

- Si priorizamos **no etiquetar injustamente** a no reincidentes, *Clean* es mejor (FPR ↓, TNR ↑).  
- Si priorizamos **detectar más reincidentes**, la pérdida es **pequeña** (TPR −0,8 p.p.), pero existe.  
- En conjunto, el comportamiento de *Clean* es **ligeramente más equilibrado** al umbral estándar **≥5**.
---


<div align="center">
<h2>Matriz de confusión y métricas (umbral ≥ 8)</h2>

In [22]:
# === 1) Calcular métricas y CMs para ambos datasets (umbral ≥5) ===
res_propu = get_cm_and_metrics(df_twoyears,       threshold=8, label='ProPublica')
res_clean = get_cm_and_metrics(df_twoyears_clean, threshold=8, label='Clean')

# === 2A) TABLA-RESUMEN DE MÉTRICAS (profesional) ===
def format_pct(x): 
    return f"{100*x:.1f}%" if pd.notnull(x) else "—"

summary = pd.DataFrame({
    'Métrica': ['Accuracy', 'TPR (Recall)', 'FPR', 'TNR (Especificidad)', 'Precision (PPV)', 'F1', 'BalancedAcc'],
    'ProPublica': [res_propu['metrics'][k] for k in ['Accuracy','TPR','FPR','TNR','Precision','F1','BalancedAcc']],
    'Clean':      [res_clean['metrics'][k] for k in ['Accuracy','TPR','FPR','TNR','Precision','F1','BalancedAcc']],
})
summary['Δ (Clean − ProPublica)'] = summary['Clean'] - summary['ProPublica']

# Formateo a texto para Table
summary_fmt = summary.copy()
for col in ['ProPublica','Clean','Δ (Clean − ProPublica)']:
    summary_fmt[col] = summary_fmt[col].map(format_pct)

table_fig = go.Figure(data=[go.Table(
    header=dict(values=list(summary_fmt.columns), fill_color="#fc9278", align='left'),
    cells=dict(values=[summary_fmt[c] for c in summary_fmt.columns], align='left')
)])
table_fig.update_layout(title="Resumen de métricas a umbral ≥8")
table_fig.show()

# === 2B) HEATMAPS DE MATRIZ DE CONFUSIÓN (lado a lado) ===
# Texto por celda: cuenta y porcentaje fila
def cm_text(cm):
    row_sums = cm.sum(axis=1, keepdims=True)
    pct = np.divide(cm, np.where(row_sums==0, 1, row_sums), where=row_sums!=0)
    return np.array([[f"{cm[0,0]}\n({pct[0,0]*100:.1f}%)", f"{cm[0,1]}\n({pct[0,1]*100:.1f}%)"],
                     [f"{cm[1,0]}\n({pct[1,0]*100:.1f}%)", f"{cm[1,1]}\n({pct[1,1]*100:.1f}%)"]])

labels_x = ['Pred: Sí', 'Pred: No']
labels_y = ['Real: Sí', 'Real: No']

fig_cm = make_subplots(rows=1, cols=2, subplot_titles=(f"Matriz - ProPublica (N={res_propu['support']['N']})",
                                                       f"Matriz - Clean (N={res_clean['support']['N']})"))

fig_cm.add_trace(go.Heatmap(
    z=res_propu['cm'], x=labels_x, y=labels_y, colorscale="GnBu", showscale=False,
    text=cm_text(res_propu['cm']), texttemplate="%{text}", hovertemplate="%{y} / %{x}<br>Casos: %{z}<extra></extra>"
), row=1, col=1)

fig_cm.add_trace(go.Heatmap(
    z=res_clean['cm'], x=labels_x, y=labels_y, colorscale='GnBu', showscale=False,
    text=cm_text(res_clean['cm']), texttemplate="%{text}", hovertemplate="%{y} / %{x}<br>Casos: %{z}<extra></extra>"
), row=1, col=2)

aplicar_estilo(fig_cm, legend_title='')
fig_cm.update_xaxes(showgrid=False)
fig_cm.update_yaxes(showgrid=False)
fig_cm.update_layout(title="Matrices de confusión a umbral ≥8")
fig_cm.show()

# === 2C) BARRAS COMPARATIVAS DE MÉTRICAS CLAVE ===
plot_metrics = ['Accuracy', 'TPR', 'FPR', 'TNR', 'BalancedAcc']
tidy = (pd.DataFrame({
    'metric': plot_metrics,
    'ProPublica': [res_propu['metrics'][m] for m in plot_metrics],
    'Clean':      [res_clean['metrics'][m] for m in plot_metrics]
})
.melt(id_vars='metric', var_name='dataset', value_name='value'))

fig_bar = px.bar(tidy, x='metric', y='value', color='dataset', barmode='group',
                 title='Comparativa de métricas a umbral ≥8', text=tidy['value'].map(lambda v: f"{v*100:.1f}%"))
fig_bar.update_traces(textposition='outside')
fig_bar.update_yaxes(tickformat=".0%")
fig_bar = aplicar_estilo(fig_bar, x_title="Métrica", y_title="Valor")
fig_bar.show()


**Qué cambia al pasar de ≥5 → ≥8 (dentro de cada dataset)**  
- **FPR** cae en seco (ProPublica **32,3%→10,1%**, Clean **30,0%→8,7%**) ⇒ **muchos menos falsos positivos**.  
- **TPR (recall)** también cae fuerte (ProPublica **62,8%→30,8%**, Clean **61,8%→30,4%**) ⇒ **se nos escapan más** verdaderos positivos.  
- **Precision (PPV)** sube (ProPublica **61,4%→71,3%**, Clean **54,5%→67,0%**) ⇒ entre los marcados “alto riesgo” hay **más** que realmente reinciden.  
- **BalancedAcc** baja (ProPublica **65,1%→60,3%**, Clean **65,9%→60,9%**) por la caída del recall.  
- **Accuracy**: baja en ProPublica (**65,4%→63,2%**) y **sube** en Clean (**67,0%→68,9%**).

**Comparando datasets al umbral ≥8 (Clean − ProPublica)**  
- **Accuracy** **+5,7 p.p.** a favor de *Clean*.  
- **TPR** prácticamente igual (**−0,4 p.p.**).  
- **FPR** **−1,4 p.p.** en *Clean* (menos falsos positivos).  
- **TNR** **+1,4 p.p.** en *Clean*.  
- **Precision (PPV)** **−4,3 p.p.** en *Clean* (coherente con su menor prevalencia).  
- **F1** **−1,2 p.p.** en *Clean*.  
- **BalancedAcc** **+0,6 p.p.** en *Clean* (ligera ventaja).

**Lectura sencilla**  
- Con **≥8** nos volvemos **más conservadores**: **evitamos muchos FP**, pero **perdemos bastantes TP**.  
- A este corte, *Clean* sigue destacando en **FPR/TNR** (menos FP, más especificidad) y logra **mejor Accuracy**, mientras que **Precision y F1** quedan algo **por debajo** de ProPublica.

---

### **CONLUSIÓN CONJUNTA** (≥5 vs ≥8)

- **≥5 (estándar)**: mejor **equilibrio** general. En *Clean* ya veíamos **FPR↓**, **TNR↑** y **BalancedAcc** algo mayor, con **pequeña** caída de **TPR/F1**.  
- **≥8 (sensibilidad)**: refuerza el perfil **conservador** (FP↓ mucho, TPR↓ mucho). *Clean* mantiene la ventaja en **FPR/TNR** y **Accuracy**, pero **PPV/F1** quedan por debajo de ProPublica.

**Decisión recomendada**  
- Reportamos **≥5** como corte principal (comparabilidad con la literatura y mejor balance).  
- Usamos **≥8** como **análisis de sensibilidad** y lo elegimos solo si el objetivo de política es **minimizar falsos positivos** por encima del recall.
---


<hr style="border:none; height:2px; background:linear-gradient(to right, #3498db, #e74c3c, #f39c12);">

<hr style="border:none; height:1px; background:#ddd;">

<div style="text-align:center; margin:20px 0;">
- • • • •
</div>

# **6. ANALISIS DE SESGOS COMPARATIVO**

In [28]:
# ===== Configuración =====
UMBRAL   = 5                      # pon 8 y vuelve a ejecutar para sensibilidad
Y_COL    = 'reincide'
SCORE    = 'resultado_compas'
RACE_COL = 'etnia'
SEX_COL  = 'genero'
AGE_COL  = 'rango_edad'          # ya categórica (p. ej. "less than 25", "25-45", "greater than 45")

# ===== Métricas estilo Barstein (8 métricas) =====
def safe_div(a,b): return a/b if b else np.nan

def metrics_barstein(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=[0,1]).ravel()
    N   = tn + fp + fn + tp
    acc = safe_div(tp+tn, N)
    fpr = safe_div(fp, fp+tn)
    fnr = safe_div(fn, fn+tp)
    prev= safe_div(tp+fn, N)       # % que realmente reinciden
    ppv = safe_div(tp, tp+fp)      # valor predictivo positivo
    npv = safe_div(tn, tn+fn)      # valor predictivo negativo
    det = safe_div(tp+fp, N)       # % etiquetado como “alto riesgo”
    return dict(N=N, Accuracy=acc, FPR=fpr, FNR=fnr,
                Prevalence=prev, PPV=ppv, NPV=npv, DetectionRate=det)

# ===== Tabla simple por grupo (sin Δ) =====
def build_group_table_simple(group_col, title, umbral=UMBRAL):
    if group_col not in df_twoyears.columns or group_col not in df_twoyears_clean.columns:
        print(f"[Aviso] La columna '{group_col}' no existe en ambos datasets.")
        return

    # Categorías presentes (tal cual, sin forzar)
    cats = sorted(set(df_twoyears[group_col].dropna().unique())
                  | set(df_twoyears_clean[group_col].dropna().unique()))
    # Orden fijo si es rango de edad y coincide con tus 3 etiquetas
    if group_col == AGE_COL:
        desired = ['less than 25','25-45','greater than 45']
        if set(cats).issubset(set(desired)):
            cats = [c for c in desired if c in cats]

    rows = []
    for g in cats:
        da = df_twoyears.loc[df_twoyears[group_col]==g, [Y_COL, SCORE]].dropna()
        dc = df_twoyears_clean.loc[df_twoyears_clean[group_col]==g, [Y_COL, SCORE]].dropna()

        ya = da[Y_COL].astype(int).values
        pa = (da[SCORE] >= umbral).astype(int).values
        yc = dc[Y_COL].astype(int).values
        pc = (dc[SCORE] >= umbral).astype(int).values

        ma = metrics_barstein(ya, pa) if len(da) else {k: np.nan for k in ['N','Accuracy','FPR','FNR','Prevalence','PPV','NPV','DetectionRate']}
        mc = metrics_barstein(yc, pc) if len(dc) else {k: np.nan for k in ['N','Accuracy','FPR','FNR','Prevalence','PPV','NPV','DetectionRate']}

        rows.append({
            'Grupo': group_col, 'Categoría': g,
            'N (ProPublica)': ma['N'], 'N (Clean)': mc['N'],
            'Prevalence (ProPublica)': ma['Prevalence'], 'Prevalence (Clean)': mc['Prevalence'],
            'Detection Rate (ProPublica)': ma['DetectionRate'], 'Detection Rate (Clean)': mc['DetectionRate'],
            'FPR (ProPublica)': ma['FPR'], 'FPR (Clean)': mc['FPR'],
            'FNR (ProPublica)': ma['FNR'], 'FNR (Clean)': mc['FNR'],
            'PPV (ProPublica)': ma['PPV'], 'PPV (Clean)': mc['PPV'],
            'NPV (ProPublica)': ma['NPV'], 'NPV (Clean)': mc['NPV'],
            'Accuracy (ProPublica)': ma['Accuracy'], 'Accuracy (Clean)': mc['Accuracy'],
        })

    df = pd.DataFrame(rows)
    if df.empty:
        print("No hay filas para mostrar.")
        return

    # Formateo %
    def fmt_pct(s): return s.map(lambda v: f"{v*100:.1f}%" if pd.notnull(v) else "—")
    pct_cols = [c for c in df.columns if c.endswith('(ProPublica)') or c.endswith('(Clean)')]
    pct_cols = [c for c in pct_cols if not c.startswith('N ')]
    df_fmt = df.copy()
    for c in pct_cols:
        df_fmt[c] = fmt_pct(df_fmt[c])

    # Orden de columnas compacto (sin Δ)
    order = ['Grupo','Categoría',
             'N (ProPublica)','N (Clean)',
             'Prevalence (ProPublica)','Prevalence (Clean)',
             'Detection Rate (ProPublica)','Detection Rate (Clean)',
             'FPR (ProPublica)','FPR (Clean)',
             'FNR (ProPublica)','FNR (Clean)',
             'PPV (ProPublica)','PPV (Clean)',
             'NPV (ProPublica)','NPV (Clean)',
             'Accuracy (ProPublica)','Accuracy (Clean)']
    df_fmt = df_fmt[[c for c in order if c in df_fmt.columns]]

    # Tabla clara con zebra-striping
    zebra = [['#f7fbff' if i%2 else '#eef5fb' for i in range(len(df_fmt))]]
    fig = go.Figure(data=[go.Table(
        header=dict(values=list(df_fmt.columns), fill_color="#fc9278", align='left', font=dict(color='black')),
        cells=dict(values=[df_fmt[c] for c in df_fmt.columns], align='left', fill_color=zebra)
    )])
    fig.update_layout(title=title, margin=dict(l=30,r=30,t=50,b=20))
    fig.show()

# ===== Ejecuta por grupo =====
build_group_table_simple(RACE_COL, f"Comparativa por RAZA/ETNIA (umbral ≥{UMBRAL})")
build_group_table_simple(SEX_COL,  f"Comparativa por GÉNERO (umbral ≥{UMBRAL})")
build_group_table_simple(AGE_COL,  f"Comparativa por RANGOS DE EDAD (umbral ≥{UMBRAL})")


### Sesgo por grupos (umbral ≥5): African-American vs Caucasian

**Qué estamos mirando.**  
Comparamos cómo se equivoca COMPAS en **dos grupos** (African-American y Caucasian), primero con el dataset original de ProPublica y luego con nuestro **Clean**. Usamos el corte estándar **≥5** para “alto riesgo” y leemos cuatro cosas muy sencillas:

- **FPR (falsos positivos)**: gente que **no** reincide pero COMPAS marcó “alto riesgo”.  
- **FNR (falsos negativos)**: gente que **sí** reincide pero COMPAS marcó “bajo riesgo”.  
- **Detection rate**: % de personas que COMPAS **etiqueta como “alto riesgo”**.  
- **Prevalence**: % que **realmente reincide** (la realidad observada a dos años).

---

#### Lo que vemos en **ProPublica**
- **FPR** es **mucho más alto en African-American** que en Caucasian  
  → COMPAS **sobremarca** “alto riesgo” a personas African-American que luego **no** reinciden.  
- **FNR** es **más alto en Caucasian**  
  → COMPAS **deja escapar** más Caucasian que **sí** reinciden (los marcó “bajo riesgo”).  
- **Detection rate vs Prevalence**  
  - African-American: el **detection rate** está **por encima** de su **prevalence** → **se etiqueta de más**.  
  - Caucasian: el **detection rate** está **por debajo** de su **prevalence** → **se etiqueta de menos**.

**Traducción llana:** el sistema **no se equivoca igual** en los dos grupos: comete **más falsos positivos** con African-American y **más falsos negativos** con Caucasian. Eso ya es una señal de **sesgo** (no cumple “errores parecidos entre grupos”).

---

#### Qué cambia en **Clean**
- **Bajamos un poco FPR** en ambos grupos (mejor para no castigar a quien no reincide),  
  pero **la brecha** sigue siendo **grande**: African-American sigue teniendo un **FPR mucho mayor** que Caucasian (en nuestra tabla ronda **~42% vs ~22%**).  
- **FNR** en Caucasian **sigue siendo alto** (incluso sube ligeramente), así que seguimos **dejando escapar** más reincidentes en ese grupo.  
- **Detection rate** se acerca más a la realidad en Caucasian y sigue siendo **alto** en African-American, lo que encaja con lo anterior.

**Conclusión práctica:** con la limpieza **mejoramos algo** (menos falsos positivos en general), pero **no eliminamos** la **asimetría de errores** entre African-American y Caucasian. COMPAS **tiende a etiquetar de más** como “alto riesgo” a African-American y **tiende a etiquetar de menos** a Caucasian.

---

#### Cómo lo usamos
- Si nuestra prioridad es **evitar falsos positivos** (no marcar “alto riesgo” a quien no reincidirá), debemos vigilar el **FPR** en African-American y quizá **subir el umbral** (ver análisis con **≥8**).  
- Si nuestra prioridad es **no dejar escapar reincidentes**, debemos vigilar el **FNR/Recall** en Caucasian y quizá **bajar el umbral** o **usar otro modelo**.

> **Idea clave:** el sesgo aquí no es “una media más alta o más baja”, sino **qué tipo de error** hace el sistema **en cada grupo**. En nuestros datos, **FPR ↑ en African-American** y **FNR ↑ en Caucasian** tanto en ProPublica como en Clean (aunque algo suavizado tras la limpieza).


<hr style="border:none; height:2px; background:linear-gradient(to right, #3498db, #e74c3c, #f39c12);">

<hr style="border:none; height:1px; background:#ddd;">

<div style="text-align:center; margin:20px 0;">
- • • • •
</div>

# 7. **CONCLUSIONES DEL ANÁLISIS COMPARATIVO DEL DATASET FILTRADO VS PROPUBLICA**



## 7.1. Qué hemos hecho y por qué

En este TFM hemos trabajado con el dataset **two_years de ProPublica**, que es la base de todos nuestros análisis.  
De forma paralela realizamos un **ejercicio de filtrado metodológico** con el objetivo de comprobar cómo afectan los criterios de coherencia temporal (ej. días entre arresto y test COMPAS, ventana de seguimiento a dos años, etc.) a los resultados de reincidencia y sesgo.


**Por qué hicimos este análisis paralelo**
- ProPublica proponía usar el dataset tal cual, pero incluía casos con incoherencias temporales.  
- Decidimos comprobar qué ocurría si aplicábamos un **filtro más estricto**, siguiendo recomendaciones metodológicas (±30 días entre arresto y evaluación, ventana ≤730 días, corte máximo en 2014).  
- Este paso **no estaba pedido en la consigna del proyecto**, pero lo incorporamos como valor añadido de calidad analítica.

Así que, partimos del *two-year dataset* de ProPublica (7.214 filas) y comprobamos que incluía casos incoherentes con un estudio “a dos años” (ej. evaluaciones COMPAS hechas mucho antes o mucho después del arresto).  
Aplicamos filtros para asegurar coherencia temporal:

- **±30 días** entre arresto y evaluación COMPAS.  
- **≤ 730 días** desde COMPAS hasta el evento de interés (dos años).  
- **Corte de fecha**: ≤ 01/04/2014 para asegurar dos años de observación.  

El filtro “días entre arrestos ≤ 730” lo analizamos, pero descartamos porque reducía demasiado la muestra y sesgaba la representatividad.

---

## 7.2. Efecto en la tasa de reincidencia

| Dataset                                | Total casos | % No reincide | % Reincide |
|----------------------------------------|-------------|---------------|------------|
| ProPublica (2016, two-year)            | 7.214       | 54,9%         | **45,1%**  |
| Barstein (2019, “corrected”)           | 6.216       | 63,8%         | **36,2%**  |
| **Nuestro filtrado (±30, ≤730, ≤1/4/14)** | ≈5.259       | **63,2%**     | **36,8%**  |

**Conclusión:** ProPublica sobreestimaba la reincidencia en **+8,3 p.p.** (≈22,6% relativo). Nuestro filtrado devuelve cifras alineadas con Barstein.

Además, los casos excluidos mostraban un **67,3% de reincidencia** frente a **36,8%** en los incluidos. Esto demuestra que la “suciedad temporal” inflaba artificialmente las tasas.

---

## 7.3. Relación riesgo COMPAS vs reincidencia real
- Se mantiene una **relación positiva y fuerte** en ambos datasets.  
- En ProPublica aparecía una correlación perfecta (*Spearman rho=1,0*) por raza, poco creíble.  
- En el dataset filtrado baja a **rho≈0,886 (p≈0,019)**: sigue siendo alta, pero más realista.  
- Por deciles de riesgo, la reincidencia real en el filtrado es sistemáticamente más baja, especialmente en los deciles 5–6.

---

## 7.4. Impacto en métricas de predicción
Probamos dos umbrales habituales para definir “alto riesgo”:

- **≥5 (umbral principal):**
  - Menos falsos positivos (FPR menor).  
  - Ligeras caídas en TPR y F1.  
  - Balanced Accuracy similar o algo mejor que en ProPublica.  

- **≥8 (sensibilidad):**
  - Reduce aún más los falsos positivos.  
  - Baja el recall (TPR) y F1.  

👉 Decisión: trabajar con **≥5 como umbral principal** y **≥8 como análisis de sensibilidad**.

---

## 7.5. Sesgo por grupos
- El sesgo se observa sobre todo en los **errores de clasificación**.  
- Con **≥5**, el patrón persiste: **African-American** tiene más falsos positivos que **Caucasian**, aunque menos marcado que en ProPublica.  
- Ajustar el umbral permite equilibrar:  
  - **≥8** → reduce falsos positivos (beneficia a African-American).  
  - **≥5** → mantiene recall más alto (beneficia a Caucasian).

---

## 7.6. Limitaciones y buenas prácticas
- Variables con pocos niveles (ej. género=2, edad=3) limitan la utilidad de pruebas como Spearman.  
- El etiquetado de “reincidencia” puede contener ruido, pero las diferencias observadas son robustas.  
- Documentamos filtros, casos excluidos y efectos en métricas para asegurar reproducibilidad.

---

## 7.7. Qué nos llevamos
1. La **coherencia temporal importa**: tras aplicar filtros, los resultados son más fiables.  
2. ProPublica **sobreestimaba la reincidencia** (~45% vs ~36%).  
3. El filtrado reduce **falsos positivos**, lo cual es clave en términos éticos y judiciales.  
4. El **sesgo por raza persiste pero se atenúa**, lo que confirma que el sesgo no se explica solo por “suciedad” en los datos.  
5. En el TFM no hemos usado el **dataset filtrado** con reportes en **≥5 y ≥8**, explicando claramente los *trade-offs*.

---

*Al imponer coherencia temporal, bajamos la tasa de reincidencia a su valor real (~36%), reducimos falsos positivos y obtenemos una lectura más justa y defendible de los sesgos en COMPAS.*


## 7.8. Decisión para el TFM
- Aunque el filtrado aporta un enfoque metodológico interesante, **decidimos mantener como base el dataset two_years de ProPublica**, ya que:
  - Es el que se nos pedía trabajar en la consigna del proyecto.  
  - Permite reproducir y discutir directamente los hallazgos de ProPublica, que son el punto de partida del debate público.  

- El análisis filtrado se presenta como un **anexo metodológico**, que:
  - Refuerza la validez de nuestras conclusiones.  
  - Muestra capacidad crítica frente a los datos.  
  - Demuestra que la tasa real de reincidencia probablemente es menor, pero que el sesgo por grupos **persiste incluso con datos corregidos**.

---
**EN RESUMEN**
1. El dataset de ProPublica (two_years) es válido para reproducir sus análisis y compararnos con su trabajo.  
2. El filtrado muestra que parte de la sobreestimación de reincidencia proviene de incoherencias temporales en los datos.  
3. Aunque las cifras cambian, el **sesgo por raza sigue existiendo**, lo que refuerza la importancia de estudiarlo.  
4. Nuestra elección metodológica equilibra **seguir la consigna** y a la vez aportar un análisis **crítico y riguroso**.

---

*Trabajamos con el dataset *two_years* de ProPublica como base del proyecto, pero mostramos con un análisis paralelo que su sobreestimación de la reincidencia se debe a incoherencias temporales; aun así, el sesgo racial persiste, lo que valida el foco de nuestro TFM.*